In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import os

def import_from_excel(e_file, filename):
    kinase_targets = e_file['kinase target'].tolist()
    if kinase_targets != []:
        kinase_targets = kinase_targets[0]#just the top score
    JS_scores = e_file['JS Distance Score'].tolist() #just the top score
    if JS_scores != []:
        JS_scores = JS_scores[0]
    #off_targets = e_file['off-target %inhibition values'].tolist()
    mu = int(filename.split('.')[0].split('_')[-2])
    return [mu, kinase_targets, JS_scores] #off_targets]

test_results_200 = []
test_results_700 = []

test_results = {}

for filename in os.listdir(os.getcwd()):
    if filename[0:8] == '030822_i':
        f = './'+str(filename)
        replicate_results = []
        targets = ''
        for x in range(1,4):
            open_file = pd.read_excel(f, str(x), engine='openpyxl')
            inhibitor_combination = import_from_excel(open_file, filename)
            targets = inhibitor_combination[1]
            replicate_results.append((x,inhibitor_combination))
        if targets != []:
            if targets not in test_results:
                test_results[targets] = [[],[]]
            if replicate_results[0][1][0] == 200:
                test_results[targets][0].append(replicate_results)
                #test_results_200.append(replicate_results)
            elif replicate_results[0][1][0] == 700:
                test_results[targets][1].append(replicate_results)
                #test_results_700.append(replicate_results)
        
#check to make sure we have 5 sets of results for each  
for k in test_results:
    print(len(test_results[k][0]))
    print(len(test_results[k][1]))

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
0
3
0
3
0
3


In [14]:
#sig results

sig_results = {}

for k in test_results:
    res_200 = test_results[k][0]
    res_700 = test_results[k][1]
    
    #get the best i=1 or i=2 score to compare to... either the most selective single inhibitor
    #or the most selective single inhibitor for each kinase
    
    #get the average i=1 score (most selective )
    i_1_200_vals = [r[0][1][2] for r in res_200 if r[0][0] == 1]
    i_1_700_vals = [r[0][1][2] for r in res_700 if r[0][0] == 1]
    
    i_2_200_vals = [r[1][1][2] for r in res_200 if r[1][0] == 2]
    i_2_700_vals = [r[1][1][2] for r in res_700 if r[1][0] == 2]
        
    sig_200_scores = []
    sig_700_scores = []
    
    sig_dist_200 = 0
    sig_dist_700 = 0
    
    if res_200 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_200[0])):
            
            i_vals = [r[i][1][2] for r in res_200]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_200_vals) > np.mean(i_2_200_vals):
                test_set = i_1_200_vals
            else:
                test_set = i_2_200_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                #if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                if np.mean(test_set)+0.001 < np.mean(i_vals):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = (np.mean(i_vals) - np.mean(test_set))   
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = np.mean(i_vals)
                        
    if res_700 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_700[0])):
            
            i_vals = [r[i][1][2] for r in res_700]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_700_vals) > np.mean(i_2_700_vals):
                test_set = i_1_700_vals
            else:
                test_set = i_2_700_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals) - np.mean(test_set)  
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals)  
                        
    if sig_200_scores != [] and sig_700_scores != []:
        knames = ''
        if sig_700_scores != []:
            for iter in test_results[k][1][0]:
                if iter[1][1] != []:
                    knames = iter[1][1]
            knames = knames.strip('][').replace('\'', '').replace(', ', '_')
        if sig_200_scores != []:
            for iter in test_results[k][0][0]:
                if iter[1][1] != []:
                    knames = iter[1][1]
            knames = knames.strip('][').replace('\'', '').replace(', ', '_')
        sig_results[knames] = [sig_200_scores,sig_700_scores,sig_dist_200,sig_dist_700]
        
print('kinase-pair','tight-prior-score','medium-prior-score')
        
for key in sig_results:
    print(key, sig_results[key][2], sig_results[key][3])

kinase-pair tight-prior-score medium-prior-score
BUB1_TGFBR2 0.02579496065460618 0.027671384004345212
CAMK2G_TBKBP1 0.002511307295525067 0.0049119157678734204
CDK1_CDK7 0.009331763527266057 0.005841983172624521
CDK4_PRKD2 0.012782258049637507 0.007265930652939567
CIT_TESK1 0.003079183393113749 0.0016651001054657222
CLK4_IRAK4 0.0028246158017988243 0.002356037733131422
EPHA4_PLK4 0.004410891614712087 0.005742405296916386
EPHB3_LCK 0.0010110845464891272 0.0011350145190549554
EPHB3_TGFBR2 0.004440309362306882 0.0027054907494041824
EPHB4_TESK1 0.0010946973740355581 0.0010329301984567385
ERN1_STK3 0.007196150654615652 0.005392398599377679
FER_PIP4K2C 0.004681882872573762 0.006765394674019376
FER_PLK4 0.042752518938191875 0.03709966942774845
FER_PTK2B 0.03017391093498545 0.03368592992683683
FRK_PLK4 0.0016486826697239554 0.005441611748426722
LCK_CSK 0.004831138614889263 0.010148303412257365
LCK_FGR 0.002648100845722756 0.0015947113045294836
LYN_PTK6 0.016906810862312716 0.025014894471905524
